# Typed Objects

In addition to the interfaces offered by `ObjectAPIClient` and `DownloadedObject`, which provides access to Geoscience Objects in a way that is agnostic to the specific object type, the `evo-objects` package also provides a set of "typed objects" that represent specific object types.

As usual, we need to first authenticate, which we can do using the `ServiceManagerWidget` from the `evo-notebooks` package

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", cache_location="./notebook-data"
).login()

## Creating a regular 3D grid object

To create a new 'regular-3d-grid' object, we use the `Regular3DGridData` class to define all of the properties of the grid, then pass that to the `Regular3DGrid.create` class method.

The `cell_data` and `vertex_data` properties are dataframes that define the attributes of the cells and vertices respectively. Each column in the dataframe represents a different attribute.

Note the order of the data is important, and must match the order of the cells and vertices in the grid. The cells are ordered such that the x-axis varies the fastest, then y, then z. So for a grid of size (10, 10, 5), the first 10 rows of the dataframe correspond to the cells at (0, 0, 0) to (9, 0, 0), the next 10 rows correspond to (0, 1, 0) to (9, 1, 0), and so on.

In [ ]:
import numpy as np
import pandas as pd

from evo.objects.typed import Point3, Regular3DGrid, Regular3DGridData, Rotation, Size3d, Size3i

grid = Regular3DGridData(
    name="Test Grid",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    cell_data=pd.DataFrame(
        {
            "value": np.random.rand(10 * 10 * 5),
            "cat": pd.Categorical.from_codes(np.random.choice(range(4), size=10 * 10 * 5), ["a", "b", "c", "d"]),
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(11 * 11 * 6),
        }
    ),
    rotation=Rotation(90, 0, 0),
)
created_grid = await Regular3DGrid.create(manager, grid)
print(f"Created regular-3d-grid object: {created_grid.metadata.url}")

## Downloading a regular 3D grid object

To download an existing 'regular-3d-grid' object, and inspect it, you can use the `Regular3DGrid.from_reference` class method. This method takes the context and the URL of the object to download.

For the example, we will download the object we just created.

In [ ]:
grid = await Regular3DGrid.from_reference(manager, created_grid.metadata.url)

print(f"Downloaded regular-3d-grid object with name: {grid.name}")
print(f"Origin: {grid.origin}")
print(f"Size: {grid.size}")
print(f"Cell Size: {grid.cell_size}")
print(f"Rotation: {grid.rotation}")
print(f"Bounding Box: {grid.bounding_box}")

To inspect the data, `grid.cells.get_dataframe()`, and `grid.vertices.get_dataframe()` can be used to get the cell and vertex data as pandas DataFrames.

In [ ]:
await grid.cells.get_dataframe()

In [ ]:
await grid.vertices.get_dataframe()

## Updating a regular 3D grid object

To update an existing 'regular-3d-grid' object, we can modify the properties of the `Regular3DGrid` instance, and then call the `update` method to submit the changes to the Geoscience Object service. This will create a new version of the object with the updated properties.

Updating the cell or vertex data can be done by calling the `set_dataframe` method on the `cells` or `vertices` attributes respectively. This will upload the new data to the Geoscience Object service, but the object will remain the same version until the `update` method is called.

In [ ]:
print(f"Version before update: {grid.metadata.version_id}")

grid.name = "Updated Test Grid"

await grid.cells.set_dataframe(
    pd.DataFrame(
        {
            "value": np.ones(10 * 10 * 5),
        }
    )
)

# Update the object to create a new version with the modified properties
await grid.update()
print(f"Updated version: {grid.metadata.version_id}")

## Creating a PointSet object

To create a new 'pointset' object, we use the `PointSetData` class to define the point locations and attributes, then pass that to the `PointSet.create` class method.

The `locations` DataFrame must have 'x', 'y', 'z' columns for coordinates. Any additional columns will be treated as point attributes.

In [ ]:
from evo.objects.typed import PointSet, PointSetData

# Create a pointset with 100 random points
num_points = 100
pointset_data = PointSetData(
    name="Test PointSet",
    locations=pd.DataFrame(
        {
            "x": np.random.rand(num_points) * 100,
            "y": np.random.rand(num_points) * 100,
            "z": np.random.rand(num_points) * 50,
            "grade": np.random.rand(num_points) * 10,  # Example attribute
            "category": pd.Categorical.from_codes(
                np.random.choice(range(3), size=num_points), ["low", "medium", "high"]
            ),
        }
    ),
)

created_pointset = await PointSet.create(manager, pointset_data)
print(f"Created pointset object: {created_pointset.metadata.url}")

## Downloading a PointSet object

To download an existing 'pointset' object, use the `PointSet.from_reference` class method. We can then access the point locations and attributes.

In [ ]:
pointset = await PointSet.from_reference(manager, created_pointset.metadata.url)

print(f"Downloaded pointset object with name: {pointset.name}")
print(f"Number of points: {pointset.num_points}")
print(f"Bounding Box: {pointset.bounding_box}")

# Get locations and attributes as a DataFrame
await pointset.locations.get_dataframe()

## Creating a regular masked 3D grid object

A masked grid is similar to a regular grid, but it includes a boolean mask indicating which cells are "active". Only active cells have attribute values, which can significantly reduce storage when working with sub-regions of a larger grid (e.g., a mining pit or geological domain).

The `mask` is a 1D boolean array with the same length as the total number of cells in the grid. The `cell_data` should only contain rows for the active cells (where `mask=True`).

In [ ]:
from evo.objects.typed import RegularMasked3DGrid, RegularMasked3DGridData

# Create a mask where only some cells are active (e.g., alternating pattern)
mask = np.array([True, False, True, False, True] * 100, dtype=bool)  # 500 cells, 300 active
number_active = np.sum(mask)

masked_grid = RegularMasked3DGridData(
    name="Test Masked Grid",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    mask=mask,
    cell_data=pd.DataFrame(
        {
            # Only provide data for active cells
            "value": np.random.rand(number_active),
            "category": pd.Categorical.from_codes(
                np.random.choice(range(3), size=number_active), ["rock", "ore", "waste"]
            ),
        }
    ),
    rotation=Rotation(45, 0, 0),
)
created_masked_grid = await RegularMasked3DGrid.create(manager, masked_grid)
print(f"Created regular-masked-3d-grid object: {created_masked_grid.metadata.url}")
print(f"Total cells: {created_masked_grid.size.total_size}, Active cells: {created_masked_grid.cells.number_active}")

### Downloading and inspecting a masked grid

The mask can be retrieved using `cells.get_mask()`, which returns a boolean numpy array. The cell data only contains values for active cells.

In [ ]:
masked_grid = await RegularMasked3DGrid.from_reference(manager, created_masked_grid.metadata.url)

print(f"Downloaded masked grid: {masked_grid.name}")
print(f"Size: {masked_grid.size}, Active cells: {masked_grid.cells.number_active}")

# Get the mask
grid_mask = await masked_grid.cells.get_mask()
print(f"Mask shape: {grid_mask.shape}, Active count: {np.sum(grid_mask)}")

# Get the cell data (only for active cells)
cell_df = await masked_grid.cells.get_dataframe()
print(f"Cell data shape: {cell_df.shape}")
cell_df.head()

## Creating a tensor 3D grid object

A tensor grid is a 3D grid where cells can have different sizes along each axis. Instead of a uniform `cell_size`, you provide arrays of cell sizes for each axis (`cell_sizes_x`, `cell_sizes_y`, `cell_sizes_z`).

This is useful for grids with variable resolution, such as grids that are finer in areas of interest and coarser elsewhere.

In [ ]:
from evo.objects.typed import Tensor3DGrid, Tensor3DGridData

# Define variable cell sizes along each axis
# X-axis: 10 cells with varying sizes (finer in the middle)
cell_sizes_x = np.array([5.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 5.0])
# Y-axis: 8 cells with uniform sizes
cell_sizes_y = np.array([2.0] * 8)
# Z-axis: 5 cells with increasing sizes (finer at top)
cell_sizes_z = np.array([1.0, 1.5, 2.0, 2.5, 3.0])

size = Size3i(len(cell_sizes_x), len(cell_sizes_y), len(cell_sizes_z))
num_cells = size.total_size
num_vertices = (size.nx + 1) * (size.ny + 1) * (size.nz + 1)

tensor_grid = Tensor3DGridData(
    name="Variable Resolution Grid",
    origin=Point3(100, 200, 0),
    size=size,
    cell_sizes_x=cell_sizes_x,
    cell_sizes_y=cell_sizes_y,
    cell_sizes_z=cell_sizes_z,
    cell_data=pd.DataFrame(
        {
            "grade": np.random.rand(num_cells) * 10,
            "density": np.random.rand(num_cells) * 2 + 2,
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(num_vertices) * 100,
        }
    ),
)
created_tensor_grid = await Tensor3DGrid.create(manager, tensor_grid)
print(f"Created tensor-3d-grid object: {created_tensor_grid.metadata.url}")
print(f"Grid extent: X={np.sum(cell_sizes_x):.1f}, Y={np.sum(cell_sizes_y):.1f}, Z={np.sum(cell_sizes_z):.1f}")

### Downloading and inspecting a tensor grid

The cell sizes for each axis can be accessed via the `cell_sizes_x`, `cell_sizes_y`, and `cell_sizes_z` properties.

In [ ]:
tensor_grid = await Tensor3DGrid.from_reference(manager, created_tensor_grid.metadata.url)

print(f"Downloaded tensor grid: {tensor_grid.name}")
print(f"Origin: {tensor_grid.origin}")
print(f"Size: {tensor_grid.size}")
print(f"Bounding Box: {tensor_grid.bounding_box}")
print(f"\nCell sizes X: {tensor_grid.cell_sizes_x}")
print(f"Cell sizes Y: {tensor_grid.cell_sizes_y}")
print(f"Cell sizes Z: {tensor_grid.cell_sizes_z}")

In [ ]:
# Get the cell and vertex data
cell_df = await tensor_grid.cells.get_dataframe()
vertex_df = await tensor_grid.vertices.get_dataframe()

print(f"Cell data: {cell_df.shape[0]} cells with columns {list(cell_df.columns)}")
print(f"Vertex data: {vertex_df.shape[0]} vertices with columns {list(vertex_df.columns)}")
cell_df.head()

## Working with Block Models

Block models are stored in the Block Model Service, which is separate from the Geoscience Object Service. The `BlockModel` class provides a unified interface that handles both services - it creates the block model in the Block Model Service and automatically creates a corresponding Geoscience Object reference.

### Creating a Regular Block Model

To create a new block model, use `BlockModel.create_regular()` with a `RegularBlockModelData` object:


In [ ]:
import numpy as np
import pandas as pd
from evo.objects.typed import BlockModel, RegularBlockModelData, Point3, Size3i, Size3d
from evo.blockmodels import Units

# Create sample block model data
# Define the grid parameters
origin = (0, 0, 0)
n_blocks = (10, 10, 5)
block_size = (2.5, 5.0, 5.0)
total_blocks = n_blocks[0] * n_blocks[1] * n_blocks[2]

# Generate block centroid coordinates (x, y, z)
# Centroids are at the center of each block
centroids = []
for k in range(n_blocks[2]):
    for j in range(n_blocks[1]):
        for i in range(n_blocks[0]):
            # Calculate centroid position: origin + (index + 0.5) * block_size
            x = origin[0] + (i + 0.5) * block_size[0]
            y = origin[1] + (j + 0.5) * block_size[1]
            z = origin[2] + (k + 0.5) * block_size[2]
            centroids.append((x, y, z))

# Create DataFrame with x, y, z coordinates (more user-friendly than i, j, k)
block_data = pd.DataFrame({
    "x": [c[0] for c in centroids],
    "y": [c[1] for c in centroids],
    "z": [c[2] for c in centroids],
    "grade": np.random.rand(total_blocks) * 10,  # Random grade values 0-10
    "density": np.random.rand(total_blocks) * 2 + 2,  # Random density 2-4
})

# Create the block model using BlockModel.create_regular()
# Use the Units class for valid unit IDs
bm_data = RegularBlockModelData(
    name="Example Block Model 26",
    description="A sample block model created from the notebook",
    origin=Point3(x=origin[0], y=origin[1], z=origin[2]),
    n_blocks=Size3i(nx=n_blocks[0], ny=n_blocks[1], nz=n_blocks[2]),
    block_size=Size3d(dx=block_size[0], dy=block_size[1], dz=block_size[2]),
    cell_data=block_data,
    crs="EPSG:28354",
    size_unit_id=Units.METRES,
    units={"grade": Units.GRAMS_PER_TONNE, "density": Units.TONNES_PER_CUBIC_METRE},
)

# Create the block model - this handles both Block Model Service and Geoscience Object creation
block_model = await BlockModel.create_regular(manager, bm_data)

print(f"Created block model: {block_model.name}")
print(f"Block Model UUID: {block_model.block_model_uuid}")
print(f"Geometry type: {block_model.geometry.model_type}")
print(f"Origin: {block_model.geometry.origin}")
print(f"N blocks: {block_model.geometry.n_blocks}")
print(f"Block size: {block_model.geometry.block_size}")
print(f"Attributes: {[attr.name for attr in block_model.attributes]}")

### Loading an Existing Block Model

You can load an existing `BlockModel` using `from_reference`, just like other typed objects:

In [ ]:
# Load the block model we just created using its URL
loaded_bm = await BlockModel.from_reference(manager, block_model.metadata.url)

print(f"Loaded BlockModel: {loaded_bm.name}")
print(f"Block Model UUID: {loaded_bm.block_model_uuid}")
print(f"Bounding Box: {loaded_bm.bounding_box}")

### Accessing Block Model Data

The `BlockModel` provides methods to access the actual block data from the Block Model Service:

In [ ]:
# Get all block data as a DataFrame
df = await block_model.get_data(columns=["*"])
print(f"Retrieved {len(df)} blocks")
df.head(10)

In [ ]:
# You can also query specific columns
grade_data = await block_model.get_data(columns=["grade"])
print(f"Grade statistics:")
print(grade_data["grade"].describe())

### Adding New Attributes to a Block Model

You can add new attributes to the block model. This creates a new version in the Block Model Service:

In [ ]:
# Calculate a new attribute based on existing data
# Get current data including coordinates
df = await block_model.get_data(columns=["x", "y", "z", "grade", "density"])

# Create new attribute DataFrame with x, y, z coordinates
df_with_new_attr = pd.DataFrame({
    "x": df["x"],
    "y": df["y"],
    "z": df["z"],
    "metal_content": df["grade"] * df["density"],  # grade * density
})

# Add the new attribute to the block model
# Use Units class for valid unit IDs
new_version = await block_model.add_attribute(
    df_with_new_attr,
    attribute_name="metal_content",
    unit=Units.KG_PER_CUBIC_METRE,
)

print(f"Added attribute 'metal_content', new version: {new_version.version_id}")

In [ ]:
new_version

### Updating Multiple Attributes

For more complex updates (adding multiple columns, updating existing columns, or deleting columns), use `update_attributes`:

In [ ]:
# Create data with multiple new attributes using x, y, z coordinates
df_updates = pd.DataFrame({
    "x": df["x"],
    "y": df["y"],
    "z": df["z"],
    "classification": np.where(df["grade"] > 5, "high", "low"),
    "value_index": df["grade"] * df["density"] * 100,
})

# Add multiple new columns at once
version = await block_model.update_attributes(
    df_updates,
    new_columns=["classification", "value_index"],
)

print(f"Updated block model, new version: {version.version_id}")

# Verify the new attributes
updated_df = await block_model.get_data(columns=["*"])
print(f"Columns now available: {list(updated_df.columns)}")


## Block Model Reports

Reports provide resource estimation summaries for block models. They calculate tonnages, grades, and metal content grouped by categories (e.g., geological domains).

**Requirements for reports:**
1. Columns must have units defined
2. At least one category column for grouping results
3. Density information (column or fixed value)

**Key classes:**
- `ReportColumnSpec` - Define which columns to report and how to aggregate them
- `ReportCategorySpec` - Define category columns for grouping
- `ReportSpecificationData` - The full report definition
- `MassUnits` - Helper with common mass unit IDs

### Add a Domain Column

First, let's add a category column for grouping. We'll create geological domains by slicing the block model into three zones based on elevation.

In [ ]:
# Refresh to get latest data
block_model = await block_model.refresh()
df = await block_model.to_dataframe()

# Create domain column based on z-coordinate (elevation)
# Divide into 3 domains: LMS1 (lower), LMS2 (middle), LMS3 (upper)
z_min, z_max = df["z"].min(), df["z"].max()
z_range = z_max - z_min

def assign_domain(z):
    if z < z_min + z_range / 3:
        return "LMS1"  # Lower zone
    elif z < z_min + 2 * z_range / 3:
        return "LMS2"  # Middle zone
    else:
        return "LMS3"  # Upper zone

df["domain"] = df["z"].apply(assign_domain)

# Add the domain column to the block model
domain_data = df[["x", "y", "z", "domain"]]
version = await block_model.add_attribute(domain_data, "domain")
print(f"Added domain column. New version: {version.version_id}")

# Check domain distribution
print("\nDomain distribution:")
print(df["domain"].value_counts())

In [ ]:
# Refresh to see the new attribute
block_model = await block_model.refresh()
block_model.attributes

### Create and Run a Report

Now we can create a report that calculates tonnages and grades by domain.

**Aggregation options (`Aggregation` enum):**
- `Aggregation.MASS_AVERAGE` - Mass-weighted average, use for **grades** (e.g., Au g/t)
- `Aggregation.SUM` - Sum of values, use for **metal content** (e.g., Au kg)

**Output unit options (`Units` class):**
- Grades: `Units.GRAMS_PER_TONNE`, `Units.PERCENT`, `Units.PPM`, `Units.OUNCES_PER_TONNE`
- Metal: `Units.KILOGRAMS`, `Units.TONNES`, `Units.GRAMS`, `Units.TROY_OUNCES`

**Mass unit options (`MassUnits` class):**
- `MassUnits.TONNES` - Metric tonnes ("t")
- `MassUnits.KILOGRAMS` - Kilograms ("kg")
- `MassUnits.OUNCES` - Troy ounces ("oz")

**Density options (choose ONE):**
- `density_column_name="density"` - Use a column (don't set `density_unit_id`)
- `density_value=2.7, density_unit_id="t/m3"` - Use fixed value (both required)

In [ ]:
from evo.blockmodels import Units
from evo.blockmodels.typed import (
    Aggregation,
    MassUnits,
    ReportSpecificationData,
    ReportColumnSpec,
    ReportCategorySpec,
)

# Define the report
report_data = ReportSpecificationData(
    name="Grade Resource Report",
    description="Resource estimate by domain",
    columns=[
        ReportColumnSpec(
            column_name="grade",
            aggregation=Aggregation.MASS_AVERAGE,  # Use MASS_AVERAGE for grades
            label="Grade",
            output_unit_id=Units.GRAMS_PER_TONNE,  # Use Units class for discoverability
        ),
        # You can add more columns:
        # ReportColumnSpec(column_name="metal", aggregation=Aggregation.SUM, label="Metal", output_unit_id=Units.KILOGRAMS),
    ],
    categories=[
        ReportCategorySpec(
            column_name="domain",
            label="Domain",
            values=["LMS1", "LMS2", "LMS3"],  # Optional: limit to specific values
        ),
    ],
    mass_unit_id=MassUnits.TONNES,  # Output mass in tonnes
    density_column_name="density",  # Use density column (unit comes from column)
    run_now=True,  # Run immediately
)

# Create the report
report = await block_model.create_report(report_data)
print(f"Created report: {report.name}")

In [ ]:
# Pretty-print the report (shows BlockSync link)
report

### View Report Results

In [ ]:
# Get the latest report result (waits if report is still running)
result = await report.refresh()

# Pretty-print the result (displays table in Jupyter)
result

In [ ]:
# Get the BlockSync URL to view the report interactively
print(f"View report in BlockSync: {report.blocksync_url}")
